In [2]:
import datetime
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn import SKLearn

import pandas as pd
import boto3 
import botocore 
import subprocess
from ast import literal_eval

from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

In [30]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()
instance_type = "ml.m4.xlarge" 

In [31]:
# POINTS TO THE FOLDER, NOT THE CSV (main_arg.py points to the actual file)

bucket = 'silverback-storage' 
train_data_key = 'train' 
test_data_key = 'test'
train_data_location = 's3://{}/{}'.format(bucket, train_data_key) 
test_data_location = 's3://{}/{}'.format(bucket, test_data_key) 

In [32]:
params = {
    "target_col": "label"
}

In [33]:
estimator = SKLearn(
    source_dir="source_dir",
    entry_point="main_arg.py",
    role=role,
    train_instance_count=1,
    train_instance_type=instance_type,
    sagemaker_session=sagemaker_session,
    base_job_name="health-nlp-test",
    py_version="py3",
    framework_version="0.20.0",
    hyperparameters=params
)

In [35]:
estimator.fit({"train": train_data_location
               , "test": test_data_location})

2019-11-18 18:12:47 Starting - Starting the training job...
2019-11-18 18:12:53 Starting - Launching requested ML instances......
2019-11-18 18:13:53 Starting - Preparing the instances for training......
2019-11-18 18:15:09 Downloading - Downloading input data
2019-11-18 18:15:09 Training - Downloading the training image..2019-11-18 18:15:27,708 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2019-11-18 18:15:27,711 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-11-18 18:15:27,723 sagemaker_sklearn_container.training INFO     Invoking user training script.
2019-11-18 18:15:27,984 sagemaker-containers INFO     Module main_arg does not provide a setup.py. 
Generating setup.py
2019-11-18 18:15:27,984 sagemaker-containers INFO     Generating setup.cfg
2019-11-18 18:15:27,984 sagemaker-containers INFO     Generating MANIFEST.in
2019-11-18 18:15:27,984 sagemaker-containers INFO     Installing module with the following c

In [36]:
predictor = estimator.deploy(initial_instance_count=1,instance_type=instance_type)

ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateEndpoint operation: The account-level service limit 'ml.m4.xlarge for endpoint usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please contact AWS support to request an increase for this limit.